# The FrameworkClient

The FrameworkClient is a client side library to allow easy interaction with the services within CAVE (connectome annotation versioning engine, also known as Dynamic Annotation Framework), eg. the annotations, stateserver. The github repository is public:
https://github.com/seung-lab/AnnotationFrameworkClient

The library can be installed directly from the github repository or from the prebuilt versions using pip:
```
pip install annotationframeworkclient
```


## Tutorials

This tutorial mainly covers the interactions with the materialized annotation tables. More information and better explanations of the other functionalities of the client can be found in the following tutorial. Please be advised that depending on your permission level you may not be able to execute all queries in this tutorial with the preset parameters as it was written with defaults for iarpa's microns project:
https://github.com/seung-lab/AnnotationFrameworkClient/blob/master/FrameworkClientExamples.ipynb


## Authentication & Authorization

If this is your first time to interact with any part of CAVE, chances are you need to setup your local credentials for your FlyWire account first. Please follow the section "Setting up your credentials" at the beginning of the tutorial above to do so.

You will need to have access to the FlyWire's production dataset to retrieve annotations. Otherwise you will see

```HTTPError: 403 Client Error: FORBIDDEN for url```

errors upon querying the materialization server.

## Initialize FrameworkClient

The FrameworkClient is instantiated with a datastack name. A datastack is a set of segmentation, and annotation tables and lives within an aligned volume (the coordinate space). FlyWire's main datastack is `flywire_fafb_production`, the aligned volume is `fafb_seung_alignment_v0` (v14.1). For convenience, there are other defaults set on the datastack level.

In [2]:
import numpy as np
import datetime
from annotationframeworkclient import FrameworkClient

In [3]:
datastack_name = "flywire_fafb_production"
client = FrameworkClient(datastack_name)

## Annotation tables

Annotations are represented by points in space and parameters (such as size, type). At specific timepoints, annotations are combined with the (proofread) segmentation to create a materialized version of the annotation table. The AnnotationEngine (`client.annotation`) owns the raw annotations and the Materialization Service (`client.materialize`) owns the materialized versions of these tables. 

To check what annotation tables are visible to you run

In [4]:
client.annotation.get_tables()

https://prod.flywire-daf.com/annotation/api/v2/aligned_volume/fafb_seung_alignment_v0/table


['synapses_nt_v1']

Every table has metadata associated with it which includes information about the owner/creator, a description and a schema that annotations in this table follow. Please review the metadata of any table you might use in the future before using it as it might contain instructions and restrictions for its usage and how to credit it's creators. For instance, the (v1) synapse table (`synapses_nt_v1`) includes an extensive description on all its columns, credits people that created it, contains instructions for citing this resource among others:

In [5]:
meta_data = client.annotation.get_table_metadata("synapses_nt_v1")
print(meta_data["description"])

FlyWire synapse description
Synapse version: 20191211
NT version: 20201223

Synapses in this table consist of a pre- and a postsynaptic point (in nm), confidence scores, and neurotransmitter information. 
The synapses were predicted by Buhmann et al [1] for the v14 alignment of the FAFB dataset. The FlyWire team remapped these synapses into the v14.1 space used by FlyWire with an accuracy of <64nm (therefore, this is a potential source of error). This version of the Buhmann et al. synapses was trained on the initial training set from the calyx and performance varies across brain areas accordingly.
Buhmann et al. assigned two scores to their synapses representing different measurements of confidence. The “connection_score” column contains the scores assigned by them during prediction (higher is more confident) and “cleft_score” contains the scores acquired by Buhmann et al. by using the synapse segmentation from Heinrich et al. [2] (higher is more confident). For the latter, Buhmann et 

The meta data contains information about the schema which ultimately determines how annotations in a table are structured. All annotations in a table follow the same schema. The synapse table follows the `fly_nt_synapse` schema:

In [6]:
meta_data["schema_type"]

'fly_nt_synapse'

## Materialized annotation tables & Queries

```
materialization = annotation + segmentation snapshot
```

As the segmentation and annotations change over time, we need to create snapshots of a combined view of them (materialized versions). Materialized versions of the annotation tables are (automatically) generated at a certain frequency. In addition to that, we are planning to include an option to retrieve any timestamp since the latest materialization ("live") but that is not available at the moment. 

There are usually a number of materialized versions available at the same time:

In [7]:
client.materialize.get_versions()

[56, 64, 65, 66, 67]

Each version comes with meta data about the time when it was created and when it will be deleted (expired). Different tables have different lifetimes and some may be LTS versions. The exact frequency and lifetime of tables will depend on how the community is using these tables. 

In [8]:
latest_version = max(client.materialize.get_versions())
client.materialize.get_version_metadata(latest_version)

{'expires_on': datetime.datetime(2021, 6, 14, 8, 10, 0, 234862),
 'version': 67,
 'valid': True,
 'datastack': 'flywire_fafb_production',
 'time_stamp': datetime.datetime(2021, 6, 7, 8, 10, 0, 234862),
 'id': 56}

Generally, specifying versions for the materialize service is optional. The latest version is used if no version is defined. 

Each materialization version contains a set of annotation tables. At the moment all tables are included in a materialization but in the future we might not include all tables in every materialization:

In [9]:
client.materialize.get_tables()

['synapses_nt_v1']

### Queries

Here, we demonstrate some queries with the synapses from Buhmann et al.. For some essential annotation types, default tables are define in the centralized info service. This way, one automatically uses the latest synapse table after an update. 

In [10]:
synapse_table = client.info.get_datastack_info()['synapse_table']
print(synapse_table)

synapses_nt_v1


Each table in this list is stored as a SQL table on the backend. The client allows users to query these tables through the frontend of the Materialization Service conventiently without the need for SQL specific language. The client will format the results as pandas dataframes. Queries are restricted to a size of 200k rows to not overwhelm the server. Should a query result in a larger list of rows, only the first 200k are returned. For bulk downloads (eg. for data preservation before a publication) please contact us.

To demonstrate this this query would pull the entire table but will only gather 200k rows (should take <2min). A warning will be raised if the query is cut short.

In [11]:
%%time

syn_df = client.materialize.query_table(synapse_table)

CPU times: user 763 ms, sys: 162 ms, total: 925 ms
Wall time: 6.88 s


In [12]:
print(len(syn_df))

200000


Here, we set the materialization version specifically. If the materialization version is not specified, the query defaults to the most recent version.

Let's take a brief look at the columns to illustrate how the materialization extends an annotation table:

In [13]:
syn_df.head()

,id,valid,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,connection_score,cleft_score,gaba,ach,glut,oct,ser,da,valid_nt,pre_pt_position,post_pt_position
0,500061,t,78606010616880715,720575940627820343,78606010616885784,720575940627820343,73.652267,0,0.061236,0.057291,1.885368e-02,0.004941,0.463699,0.393978,t,"[463832, 269164, 75320]","[463944, 269104, 75320]"
1,500123,t,82194816569875334,720575940611969366,82194816569876563,720575940614396021,372.543762,129,0.015839,0.197263,1.752463e-02,0.000123,0.331374,0.437876,t,"[674196, 269084, 74080]","[674212, 269196, 74040]"
2,520,t,82686985462268338,720575940632481933,82686985462269565,720575940615229806,611.586121,143,0.002861,0.914896,8.416596e-05,0.000317,0.000054,0.081788,t,"[700272, 245104, 74320]","[700280, 245004, 74280]"
3,1000458,t,78182767359664830,720575940617001339,78182767359678728,720575940625126931,16.290222,142,0.000365,0.912595,6.038809e-07,0.000009,0.000018,0.087012,t,"[439840, 210096, 75480]","[439832, 210208, 75400]"
4,500755,t,81983710337347499,720575940612267518,81983710337346325,720575940597072847,242.144440,143,0.002634,0.934998,5.801082e-02,0.000031,0.000619,0.003707,t,"[661648, 271776, 74040]","[661632, 271892, 74040]"


Annotations consist of parameters and spatial points. Some or all of these spatial points are what we call "BoundSpatialPoints". These are linked to the segmentation during materialization. The synapse tables have two such points (`pre_pt`, `post_pt`). Per point there are three columns: `*_position`, `*_supervoxel_id`, `*_root_id`. Supervoxels are the small atomic segments, and root ids describe large components (neurons) consisting of many supervoxels. A root id always refers to the same version of a neuron and represents a snapshot in time in its own right. For a given annotation id (`id`), all but the `*_root_id` columns stay constant between materializations. 

`query_table` has three parameters to define filters: filter_in_dict, filter_out_dict, filter_equal_dict. More options will be added. This can be used to query synapses between any lists of neurons. For instance, to query the outgoing synapses of an AMMC-B1 neuron we included in the FlyWire paper:
(see the next section for how to come up with a specific root id)

In [14]:
%%time

syn_df = client.materialize.query_table(synapse_table, 
                                        filter_in_dict={"pre_pt_root_id": [720575940627197566]})

CPU times: user 40 ms, sys: 6.5 ms, total: 46.5 ms
Wall time: 5.83 s


As described in the metadata above, we suggest filtering the synapse table using the `cleft_score` and `connection_score`. Tuning these will help to reduce the number of false positive synapses in the list. The best threshold(s) will depend on the specific neurons included in the analysis. Here we will just remove all synapses with a `cleft_score < 50`.

In [15]:
syn_df = syn_df[syn_df["cleft_score"] >= 50]

Some postsynaptic partners have a 0 id. Many of these are due to the synapse prediction covering a bigger space than the segmentation. Here, we remove these along with synapses onto itself as we are confident that this cell does not make autapses.

In [16]:
syn_df = syn_df[syn_df["pre_pt_root_id"] != syn_df["post_pt_root_id"]]
syn_df = syn_df[syn_df["post_pt_root_id"] != 0]

This synapse table comes with neurotransmitter prediction from the work of Eckstein et al.. Please review the description in the metadata to understand the caveats of this data with regards your analysis. Here, we just look at the mean of the probablities of all outgoing synapses which shows that this neuron's neurotransmitter is very likely acetylcholine.

In [17]:
np.mean(syn_df[["gaba", "ach", "glut", "oct", "ser", "da"]])

gaba    0.032069
ach     0.835793
glut    0.041500
oct     0.005178
ser     0.021450
da      0.064010
dtype: float64

Here we take a brief look at the postsynaptic partners and sorting them by number of synapses; displaying the top 10:

In [18]:
u_post_root_ids, c_post_root_ids = np.unique(syn_df["post_pt_root_id"], return_counts=True)

sorting = np.argsort(c_post_root_ids)[::-1][:10]
list(zip(u_post_root_ids[sorting], c_post_root_ids[sorting]))

[(720575940612001489, 96),
 (720575940639811469, 92),
 (720575940606297353, 88),
 (720575940615361748, 86),
 (720575940621893127, 84),
 (720575940621301738, 80),
 (720575940626312778, 73),
 (720575940625492753, 71),
 (720575940623903434, 65),
 (720575940618489734, 64)]

The main target is an AMMC-A1 (720575940613535430) which is a connection we described in Figure 6 in the FlyWire paper.

We can further restrict the query by filtering the postsynaptic targets. For instance this query will only return the synapses between the these two root ids.

In [19]:
syn_df = client.materialize.query_table(synapse_table, 
                                        filter_in_dict={"pre_pt_root_id": [720575940627197566],
                                                        "post_pt_root_id": [720575940612001489]})
syn_df = syn_df[syn_df["cleft_score"] >= 50]

syn_df

,id,valid,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,connection_score,cleft_score,gaba,ach,glut,oct,ser,da,valid_nt,pre_pt_position,post_pt_position
0,211365569,t,77832229442872849,720575940627197566,77832229442883229,720575940612001489,52.161354,100,0.050798,0.706145,0.103963,0.020856,0.007026,0.111212,t,"[418808, 286736, 110840]","[418760, 286616, 110880]"
1,217890780,t,77691423235424395,720575940627197566,77691423235425756,720575940612001489,102.692398,145,0.004396,0.970069,0.000883,0.000840,0.000148,0.023665,t,"[412236, 283056, 121640]","[412208, 283148, 121680]"
2,6180240,t,77761173637894262,720575940627197566,77761173637888123,720575940612001489,227.389908,158,0.011852,0.966436,0.005491,0.002103,0.000580,0.013538,t,"[414292, 245008, 144520]","[414160, 245016, 144520]"
3,51374154,t,77691285729116258,720575940627197566,77691285729100925,720575940612001489,111.818047,132,0.008370,0.705035,0.009202,0.000005,0.272142,0.005246,t,"[409856, 274840, 93040]","[409984, 274868, 93040]"
4,237434942,t,77832023284765928,720575940627197566,77832023284771485,720575940612001489,161.775467,141,0.023719,0.890757,0.013778,0.002228,0.019519,0.049999,t,"[417900, 272952, 119880]","[417844, 273044, 119880]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,236131087,t,77761585821121562,720575940627197566,77761585821120349,720575940612001489,88.254326,116,0.007150,0.958711,0.001355,0.016179,0.000408,0.016197,t,"[414620, 270192, 119680]","[414520, 270216, 119680]"
122,217890770,t,77691423235405859,720575940627197566,77691423235425756,720575940612001489,65.530502,145,0.003493,0.976093,0.000120,0.003406,0.000033,0.016854,t,"[412216, 283052, 121600]","[412128, 283092, 121640]"
123,147115465,t,77832023284730175,720575940627197566,77832023284716150,720575940612001489,13.454404,142,0.014081,0.748565,0.001552,0.000199,0.003485,0.232117,t,"[419512, 274800, 118480]","[419480, 274656, 118480]"
124,221946020,t,77832160723343355,720575940627197566,77832160723346346,720575940612001489,1038.665283,143,0.048305,0.895462,0.023994,0.006970,0.001213,0.024056,t,"[417700, 282552, 108800]","[417580, 282568, 108800]"


## "Live" Materialization Queries

In [23]:
import networkx as nx

In [24]:
g = client.chunkedgraph.get_lineage_graph(720575940645374596)

In [25]:
gt = nx.node_link_graph(g)

NodeView((720575940645374596, 720575940606531081, 720575940621404710, 720575940637186126, 720575940637309117, 720575940626814031, 720575940627185911))

In [72]:
%%time 

syn_df = client.materialize.live_query(synapse_table, 
                                       filter_in_dict={"pre_pt_root_id": [720575940645374596]},
                                       timestamp=datetime.datetime.utcnow())

syn_df = syn_df[syn_df["cleft_score"] >= 50]

syn_df

ValueError: [720575940645374596] are not valid rootIDs at timestamp= 2021-06-04 01:02:07.913412, 
                                use chunkedgraph client to query lineage graph to find new ID(s)

In [63]:
%%time 

syn_df = client.materialize.live_query(synapse_table, 
                                       filter_in_dict={"pre_pt_root_id": [720575940627197566],
                                                       "post_pt_root_id": [720575940612001489]},
                                       timestamp=datetime.datetime.utcnow())

syn_df = syn_df[syn_df["cleft_score"] >= 50]

syn_df

CPU times: user 30 ms, sys: 628 µs, total: 30.6 ms
Wall time: 927 ms


,id,valid,pre_pt_supervoxel_id,pre_pt_root_id,post_pt_supervoxel_id,post_pt_root_id,connection_score,cleft_score,gaba,ach,glut,oct,ser,da,valid_nt,pre_pt_position,post_pt_position
0,6162631,t,77761104918444158,720575940627197566,77761104918444188,720575940612001489,326.542755,102,0.147903,0.462820,0.375533,0.005401,0.001151,0.007191,t,"[414712, 243344, 145120]","[414600, 243276, 145120]"
1,6180240,t,77761173637894262,720575940627197566,77761173637888123,720575940612001489,227.389908,158,0.011852,0.966436,0.005491,0.002103,0.000580,0.013538,t,"[414292, 245008, 144520]","[414160, 245016, 144520]"
3,8180655,t,77761173637907153,720575940627197566,77761173637915205,720575940612001489,49.321461,176,0.016921,0.900563,0.015684,0.001348,0.005939,0.059545,t,"[415680, 247148, 144920]","[415632, 247108, 145040]"
4,40964236,t,77691492021634849,720575940627197566,77691492021615097,720575940612001489,7.644144,62,0.024671,0.912242,0.009162,0.000161,0.018725,0.035040,t,"[412888, 286088, 130840]","[412960, 286024, 130840]"
5,51374115,t,77691285729051546,720575940627197566,77691285729087258,720575940612001489,167.695801,106,0.025043,0.350947,0.025653,0.002037,0.509739,0.086582,t,"[410048, 275336, 92480]","[410000, 275244, 92520]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,236982393,t,77761585821126507,720575940627197566,77761585821131453,720575940612001489,476.344971,145,0.009036,0.962195,0.000537,0.000177,0.001508,0.026548,t,"[415872, 269404, 120080]","[415864, 269284, 120040]"
122,237434920,t,77832023284777092,720575940627197566,77832023284775770,720575940612001489,58.748398,144,0.032626,0.951942,0.001326,0.000276,0.000651,0.013178,t,"[418172, 273748, 119600]","[418084, 273656, 119560]"
123,237434941,t,77831954565287519,720575940627197566,77831954565292946,720575940612001489,37.415009,145,0.013553,0.776630,0.010634,0.002239,0.005292,0.191653,t,"[417720, 272816, 119840]","[417708, 272892, 119880]"
124,237434942,t,77832023284765928,720575940627197566,77832023284771485,720575940612001489,161.775467,141,0.023719,0.890757,0.013778,0.002228,0.019519,0.049999,t,"[417900, 272952, 119880]","[417844, 273044, 119880]"


## Retrieving matching root ids

Neuroglancer shows the most recent version of the segmentation by default. Neurons that have been updated since a materialized version are not included in a table of that version. To reconcile this, users need to look up root ids for their data with a timestamp. 

We generally recommend storing annotations as points in space as these can be mapped to root ids easily (that's basically what materialization is). Soon, users will be able to create their own annotation tables and CAVE will provide fitting root ids automatically. Still, use cases will arrive that require a mnual materialization by the user:


### Programmatically

The client has an interface to the chunkedgraph (see Section 5 in [the related tutorial](https://github.com/seung-lab/AnnotationFrameworkClient/blob/master/FrameworkClientExamples.ipynb) which allows users to query a root id for a given supervoxel id. Supervoxel ids can be retrieved from the segmentation using [cloudvolume](https://github.com/seung-lab/cloud-volume/).


### Neuroglancer

The segmentation layer has an option under the tab "graph" to lock a layer to a specific timestamps. Then, root ids are looked up with this specific timestamp (proofreading is not possible in this mode). Be aware that this mode does not prevent the pasting of root ids from different timestamps into the layer as that circumvents the lookup to the server.

### Timestamps

Timestamps are _always_ UTC. 

Please be aware that the package or browser you are using might format timestamps in your local timezone. The timestamp for all annotation tables within a materialization are the same:

In [18]:
client.materialize.get_version_metadata(15)

{'time_stamp': '2021-03-10T19:07:56.375440',
 'version': 15,
 'valid': True,
 'expires_on': '2021-04-09T19:07:56.375440',
 'datastack': 'flywire_fafb_production',
 'id': 3}

## Creating neuroglancer links programmatically

We are building infrastructure into neuroglancer to display this information there while browsing neurons. Until this is ready, the most convenient way to visualize this information in neuroglancer is to programmatically create neuroglancer state and to upload them to the state server. The links can then be distributed. 

[NeuroglancerAnnotationUI (nglui)](https://github.com/seung-lab/NeuroglancerAnnotationUI)  makes programmatic creation of neuroglancer states convenient. The [statebuilder examples](https://github.com/seung-lab/NeuroglancerAnnotationUI/blob/master/examples/statebuilder_examples.ipynb) shows how one can directly from dataframes as the one above to neuroglancer states. The [related tutorial on this client](https://github.com/seung-lab/AnnotationFrameworkClient/blob/master/FrameworkClientExamples.ipynb) shows under "4. JSON Service" how this client can be used to upload states to the server and to create neuroglancer links.


## Further references


More examples for the usage of CAVE can be found in a related project:

https://github.com/AllenInstitute/MicronsBinder

A rough overview of the structure of our backend services can be found here:

https://github.com/seung-lab/AnnotationPipelineOverview

## Credit

CAVE is developed at Princeton University and the Allen Institute for Brain Science within the iarpa MICrONs project and the FlyWire project. Main contributors to the design and backend development 
are Derrick Brittain, Forrest Collman, Sven Dorkenwald, Chris Jordan, Casey Schneider-Mizell

A citable publication is in the works. Please contact us if you are interested in using CAVE on another dataset. 